# CH2 실습

## 1. 인덱스와 슬라이스
### 1-2. 자체 시퀀스 생성

In [1]:
class Items:
    def __init__(self, *values):
        self._values = list(values)

    def __len__(self):
        return len(self._values)

    def __getitem__(self, item):
        return self._values.__getitem__(item)

In [4]:
item = Items(1,2,3)
len(item)
item.__getitem__(1)

2

## 2. 컨텍스트 관리자

In [9]:
run = print

def stop_database():
    run("systemctl stop postgresql.service")

def start_database():
    run("systemctl start postgresql.service")

class DBHandler:
    def __enter__(self):
        stop_database()
        return self
    
    def __exit__(self, exc_type, ex_value, ex_traceback):
        start_database()

def db_backup():
    run("pg_dump database")

def main():
    with DBHandler():
        db_backup()
        
if __name__ == "__main__":
    main()

systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service


In [8]:
import contextlib
@contextlib.contextmanager
def db_handler():
    stop_database()
    yield
    start_database()

with db_handler():
    db_backup()

systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service


In [12]:
class dbhandler_decorator(contextlib.ContextDecorator):
    def __enter__(self):
        stop_database()

    def __exit__(self, ext_type, ex_value, ex_traceback):
        start_database()

@dbhandler_decorator()
def offline_backup():
    run("pg_dump database")

offline_backup()

systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service


## 4. 이터러블 객체
### 4-1.이터러블 객체 만들기

In [11]:
from datetime import timedelta, date

class DateRangeIterable:
    """An iterable that contains its own iterator object."""

    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self._present_day = start_date

    def __iter__(self):
        return self

    def __next__(self):
        if self._present_day >= self.end_date:
            raise StopIteration
        today = self._present_day
        self._present_day += timedelta(days=1)
        return today


In [12]:
for day in DateRangeIterable(date(2021, 7, 6), date(2021, 7, 16)):
    print(day)

2021-07-06
2021-07-07
2021-07-08
2021-07-09
2021-07-10
2021-07-11
2021-07-12
2021-07-13
2021-07-14
2021-07-15


In [14]:
r = DateRangeIterable(date(2021, 7, 6), date(2021, 7, 16))
next(r)

datetime.date(2021, 7, 6)

In [24]:
next(r)

StopIteration: 

In [25]:
r = DateRangeIterable(date(2021, 1, 1), date(2021, 1, 5))
", ".join(map(str, r))

'2021-01-01, 2021-01-02, 2021-01-03, 2021-01-04'

In [26]:
max(r)

ValueError: max() arg is an empty sequence

In [27]:
class DateRangeContainerIterable:
    """An range that builds its iteration through a generator."""

    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date

    def __iter__(self):
        current_day = self.start_date
        while current_day < self.end_date:
            yield current_day
            current_day += timedelta(days=1)

- 각각의 for루프는 `__iter__`를 호출하고
- `__inter__`는 다시 제너레이터를 생성

In [29]:
r = DateRangeContainerIterable(date(2021, 1, 1), date(2021, 1, 5))
", ".join(map(str, r))
max(r)

datetime.date(2021, 1, 4)

### 4-2. 시퀀스 만들기

In [31]:
class DateRangeSequence:
    """An range created by wrapping a sequence."""

    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self._range = self._create_range()

    def _create_range(self):
        days = []
        current_day = self.start_date
        while current_day < self.end_date:
            days.append(current_day)
            current_day += timedelta(days=1)
        return days

    def __getitem__(self, day_no):
        return self._range[day_no]

    def __len__(self):
        return len(self._range)


In [32]:
s1 = DateRangeIterable(date(2021, 1, 1), date(2021, 1 ,5))
for day in s1:
    print(day)

2021-01-01
2021-01-02
2021-01-03
2021-01-04


> walrus operator
- [docs](https://docs.python.org/3/whatsnew/3.8.html)
- 표현식을 인라인으로 캡처하는 문법
- assigns values to variables as part of a larger expression.
- 코드가 짧아질 수는 있지만 readability가 떨어질 수 있음
- 함수화등을 할때 분할화가 힘드므로 유지보수적으로는 떨어지는 거 같다

In [37]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
if (n := len(a)) > 10:
    print(f"List is too long ({n} elements, expected <= 10)")

List is too long (11 elements, expected <= 10)


In [59]:
if n > 20:
    print(True)
else:
    print(False)

False


In [34]:
lst = [y := 2, y**2, y**3]
print(lst)

[2, 4, 8]


In [61]:
# 삼항연산자
b = 5
a = 4 if a<b else 2
print(a)

TypeError: '<' not supported between instances of 'list' and 'int'

## 8. 파이썬에서 유의할 점
### 8-1. 변경 가능한(mutable) 파라미터의 기본 값


In [40]:
def wrong_user_display(user_metadata: dict = {"name": "John", "age": 30}):
    name = user_metadata.pop("name")
    age = user_metadata.pop("age")

    return f"{name} ({age})"

def user_display(user_metadata: dict = None):
    user_metadata = user_metadata or {"name": "John", "age": 30}

    name = user_metadata.pop("name")
    age = user_metadata.pop("age")

    return f"{name} ({age})"


In [41]:
wrong_user_display()

'John (30)'

In [42]:
wrong_user_display({"name": "Jane", "age": 25})

'Jane (25)'

- 인자를 변경하면 다음에 호출할 때 명시적으로 user_metadate를 전달하지 않으면 
KeyError가 발생

In [43]:
wrong_user_display()

KeyError: 'name'

In [44]:
user_display()

'John (30)'

In [45]:
user_display({"name": "Jane", "age": 25})

'Jane (25)'

In [46]:
user_display()

'John (30)'

### 8-2. 내장(built-in) 타입 확장

In [62]:
from collections import UserList
class BadList(list):
    def __getitem__(self, index):
        value = super().__getitem__(index)
        if index % 2 == 0:
            prefix = "짝수"
        else:
            prefix = "홀수"
        return f"[{prefix}] {value}"


class GoodList(UserList):
    def __getitem__(self, index):
        value = super().__getitem__(index)
        if index % 2 == 0:
            prefix = "짝수"
        else:
            prefix = "홀수"
        return f"[{prefix}] {value}"

In [63]:
b1 = BadList((0, 1, 2, 3, 4, 5))
print(b1[0])
print(b1[5])

[짝수] 0
[홀수] 5


In [64]:
"".join(b1)

TypeError: sequence item 0: expected str instance, int found

In [65]:
g1 = GoodList((0, 1, 2, 3, 4, 5))
"".join(g1)


'[짝수] 0[홀수] 1[짝수] 2[홀수] 3[짝수] 4[홀수] 5'